In [1]:
!python -m spacy download en_core_web_md
!python -m spacy link en_core_web_md en --force;

     |████████████████████████████████| 50.8 MB 91 kB/s eta 0:00:0119   |██                              | 3.2 MB 41 kB/s eta 0:19:09     |██████████████████              | 28.6 MB 33 kB/s eta 0:11:01
You should consider upgrading via the '/home/alee/Documents/Rasa_Nlu/venv/bin/python -m pip install --upgrade pip' command.
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')
✔ Linking successful
/home/alee/Documents/Rasa_Nlu/venv/lib/python3.6/site-packages/en_core_web_md
--> /home/alee/Documents/Rasa_Nlu/venv/lib/python3.6/site-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [10]:
!python -m spacy download en

     |████████████████████████████████| 12.0 MB 759 kB/s eta 0:00:01
  Created wheel for en-core-web-sm: filename=en_core_web_sm-2.3.1-py3-none-any.whl size=12047106 sha256=bb34b532a7a6c494bff60f4c0452614b6fd227f9bd9a1d9bf45cf0a83b043aaa
  Stored in directory: /tmp/pip-ephem-wheel-cache-4iq9tpvy/wheels/10/6f/a6/ddd8204ceecdedddea923f8514e13afb0c1f0f556d2c9c3da0
Successfully built en-core-web-sm
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/home/alee/Documents/Rasa_Nlu/venv/lib/python3.6/site-packages/en_core_web_sm
--> /home/alee/Documents/Rasa_Nlu/venv/lib/python3.6/site-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [2]:
import rasa_nlu
import rasa_core
import spacy

print("rasa_nlu: {} rasa_core: {}".format(rasa_nlu.__version__, rasa_core.__version__))
print("Loading spaCy language model...")
print(spacy.load("en")("Hello world!"))

rasa_nlu: 0.15.1 rasa_core: 0.14.5
Loading spaCy language model...
Hello world!


In [3]:
from rasa_nlu.training_data import load_data
from rasa_nlu.config import RasaNLUModelConfig
from rasa_nlu.model import Trainer
from rasa_nlu import config

# loading the nlu training samples
training_data = load_data("nlu.md")

# trainer to educate our pipeline
trainer = Trainer(config.load("config.yml"))

# train the model!
interpreter = trainer.train(training_data)

# store it for future use
model_directory = trainer.persist("./models/nlu/", fixed_model_name="current")

Fitting 2 folds for each of 6 candidates, totalling 12 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/home/alee/Documents/Rasa_Nlu/venv/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/alee/Documents/Rasa_Nlu/venv/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/alee/Documents/Rasa_Nlu/venv/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/alee/Documents/Rasa_Nlu/venv/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and 

In [4]:
msg=interpreter.parse("Hey")
print(msg)

{'intent': {'name': 'greet', 'confidence': 0.5814455360822202}, 'entities': [], 'intent_ranking': [{'name': 'greet', 'confidence': 0.5814455360822202}, {'name': 'goodbye', 'confidence': 0.278271137807093}, {'name': 'mood_great', 'confidence': 0.04158815210199033}, {'name': 'mood_affirm', 'confidence': 0.034023384954805634}, {'name': 'mood_unhappy__', 'confidence': 0.032739910878631984}, {'name': 'mood_deny', 'confidence': 0.019900834938697425}, {'name': 'inform', 'confidence': 0.012031043236560825}], 'text': 'Hey'}


In [5]:
from rasa_nlu.test import run_evaluation

run_evaluation("nlu.md", model_directory)

100%|██████████| 85/85 [00:00<00:00, 118.24it/s]


{'intent_evaluation': {'predictions': [{'text': 'hey',
    'intent': 'greet',
    'predicted': 'greet',
    'confidence': 0.5814455360822202},
   {'text': 'hello there',
    'intent': 'greet',
    'predicted': 'greet',
    'confidence': 0.494232646255804},
   {'text': 'hi',
    'intent': 'greet',
    'predicted': 'greet',
    'confidence': 0.5925337681016158},
   {'text': 'hello there',
    'intent': 'greet',
    'predicted': 'greet',
    'confidence': 0.494232646255804},
   {'text': 'good morning',
    'intent': 'greet',
    'predicted': 'greet',
    'confidence': 0.5121357803195928},
   {'text': 'good evening',
    'intent': 'greet',
    'predicted': 'greet',
    'confidence': 0.5117697965329514},
   {'text': 'moin',
    'intent': 'greet',
    'predicted': 'greet',
    'confidence': 0.500475090703367},
   {'text': 'hey there',
    'intent': 'greet',
    'predicted': 'greet',
    'confidence': 0.4960047355357697},
   {'text': "let's go",
    'intent': 'greet',
    'predicted': 'greet'

In [6]:
from rasa_core.policies import FallbackPolicy, KerasPolicy, MemoizationPolicy
from rasa_core.agent import Agent

# The fallback action will be executed if the intent recognition has #a confidence below nlu_threshold or if none of the dialogue #policies predict an action with confidence higher than #core_threshold.

fallback = FallbackPolicy(fallback_action_name="utter_unclear",
                          core_threshold=0.2,
                          nlu_threshold=0.1)

agent = Agent('domain.yml', policies=[MemoizationPolicy(), KerasPolicy(), fallback])

# loading our neatly defined training dialogues
training_data = agent.load_data('stories.md')

agent.train(training_data)

agent.persist('models/dialogue')

Processed Story Blocks: 100%|██████████| 7/7 [00:00<00:00, 142.81it/s, # trackers=12]
Processed actions: 0it [00:00, ?it/s, # examples=16]


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



Processed actions: 33it [00:00, 691.58it/s, # examples=33]


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking (Masking)            (None, 5, 24)             0         
_________________________________________________________________
lstm (LSTM)                  (None, 32)                7296      
_________________________________________________________________
dense (Dense)                (None, 15)                495       
_________________________________________________________________
activation (Activation)      (None, 15)                0         
Total params: 7,791
Trainable params: 7,791
Non-trainable params: 0
_________________________________________________________________
Instructions for updating:
Use tf.cast instead.
Epoch 1/100
240/240 [================

240/240 [==============================] - 0s 280us/sample - loss: 0.5078 - acc: 0.9042
Epoch 65/100
240/240 [==============================] - 0s 288us/sample - loss: 0.5153 - acc: 0.8917
Epoch 66/100
240/240 [==============================] - 0s 225us/sample - loss: 0.4966 - acc: 0.9042
Epoch 67/100
240/240 [==============================] - 0s 246us/sample - loss: 0.5008 - acc: 0.9042
Epoch 68/100
240/240 [==============================] - 0s 272us/sample - loss: 0.4827 - acc: 0.8958
Epoch 69/100
240/240 [==============================] - 0s 246us/sample - loss: 0.4776 - acc: 0.9208
Epoch 70/100
240/240 [==============================] - 0s 257us/sample - loss: 0.4553 - acc: 0.9125
Epoch 71/100
240/240 [==============================] - 0s 282us/sample - loss: 0.4475 - acc: 0.9000
Epoch 72/100
240/240 [==============================] - 0s 239us/sample - loss: 0.4391 - acc: 0.9167
Epoch 73/100
240/240 [==============================] - 0s 269us/sample - loss: 0.4392 - acc: 0.9208
Epo

In [7]:
agent.handle_text("what is you?")

[{'recipient_id': 'default', 'text': 'Bye'}]

In [10]:
from rasa_core.agent import Agent
from rasa_core.interpreter import NaturalLanguageInterpreter
interpreter = NaturalLanguageInterpreter.create(model_directory)
agent = Agent.load("models/dialogue", interpreter=interpreter)
while True:
    a = input()
    if a == "stop":
        break
    responses = agent.handle_text(a)
    for response in responses:
        print(response['text'])


Hello
Hey! How are you?
I am Good
Great carry on!
stop


In [11]:
!pip freeze > requirements.txt